In [ ]:
# https://www.kaggle.com/code/dimitreoliveira/model-stacking-feature-engineering-and-eda

## Goal is to forecast the total amount of products sold in every shop for the test set

In [2]:
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import catboost
from catboost import Pool
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler

%matplotlib inline
sns.set(style="darkgrid")
pd.set_option('display.float_format', lambda x: '%.2f' % x)
warnings.filterwarnings("ignore")

In [38]:
sales = pd.read_csv("E:/Kaggle_Practice/Predict Future Sales/sales_train.csv",  parse_dates=['date'],dtype={'date': 'str', 'date_block_num': 'int32', 'shop_id': 'int32', 
                          'item_id': 'int32', 'item_price': 'float32', 'item_cnt_day': 'int32'})
item_cat = pd.read_csv("E:/Kaggle_Practice/Predict Future Sales/item_categories.csv")
item = pd.read_csv("E:/Kaggle_Practice/Predict Future Sales/items.csv")
sub = pd.read_csv("E:/Kaggle_Practice/Predict Future Sales/sample_submission.csv")
shops = pd.read_csv("E:/Kaggle_Practice/Predict Future Sales/shops.csv")
test = pd.read_csv("E:/Kaggle_Practice/Predict Future Sales/test.csv")

### File descriptions
#### sales_train.csv - the training set. Daily historical data from January 2013 to October 2015.
#### test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.
#### sample_submission.csv - a sample submission file in the correct format.
#### items.csv - supplemental information about the items/products.
#### item_categories.csv  - supplemental information about the items categories.
#### shops.csv- supplemental information about the shops.

### DATA provide with daily sales data for each store-item combination
### Data fields
#### ID - an Id that represents a (Shop, Item) tuple within the test set
#### shop_id - unique identifier of a shop
#### item_id - unique identifier of a product
#### item_category_id - unique identifier of item category
#### item_cnt_day - number of products sold. You are predicting a monthly amount of this measure
#### item_price - current price of an item
#### date - date in format dd/mm/yyyy
#### date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,.., October 2015 is 33
#### item_name - name of item
#### shop_name - name of shop
#### item_category_name - name of item category
#### This dataset is permitted to be used for any purpose, including commercial use.

In [4]:
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.00
1,03.01.2013,0,25,2552,899.00,1.00
2,05.01.2013,0,25,2552,899.00,-1.00
3,06.01.2013,0,25,2554,1709.05,1.00
4,15.01.2013,0,25,2555,1099.00,1.00


In [13]:
sales.shape

(2935849, 6)

In [7]:
item.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [9]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [10]:
shops.shape

(60, 2)

In [12]:
item_cat

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4
...,...,...
79,Служебные,79
80,Служебные - Билеты,80
81,Чистые носители (шпиль),81
82,Чистые носители (штучные),82


### Join data sets 

In [39]:
train = sales.join(item, on='item_id', rsuffix = '_').join(shops, on = 'shop_id', rsuffix = '_').join(item_cat, on = 'item_category_id', rsuffix = '_').drop(['item_id_', 'shop_id_', 'item_category_id_'], axis=1)

In [40]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,shop_name,item_category_name
0,2013-02-01,0,59,22154,999.00,1,ЯВЛЕНИЕ 2012 (BD),37,"Ярославль ТЦ ""Альтаир""",Кино - Blu-Ray
1,2013-03-01,0,25,2552,899.00,1,DEEP PURPLE The House Of Blue Light LP,58,"Москва ТРК ""Атриум""",Музыка - Винил
2,2013-05-01,0,25,2552,899.00,-1,DEEP PURPLE The House Of Blue Light LP,58,"Москва ТРК ""Атриум""",Музыка - Винил
3,2013-06-01,0,25,2554,1709.05,1,DEEP PURPLE Who Do You Think We Are LP,58,"Москва ТРК ""Атриум""",Музыка - Винил
4,2013-01-15,0,25,2555,1099.00,1,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,"Москва ТРК ""Атриум""",Музыка - CD фирменного производства


### Let's take a look at the raw data

In [26]:
train.shape

(2935849, 10)

In [27]:
train.head().T

,0,1,2,3,4
date,02.01.2013,03.01.2013,05.01.2013,06.01.2013,15.01.2013
date_block_num,0,0,0,0,0
shop_id,59,25,25,25,25
item_id,22154,2552,2552,2554,2555
item_price,999.00,899.00,899.00,1709.05,1099.00
item_cnt_day,1.00,1.00,-1.00,1.00,1.00
item_name,ЯВЛЕНИЕ 2012 (BD),DEEP PURPLE The House Of Blue Light LP,DEEP PURPLE The House Of Blue Light LP,DEEP PURPLE Who Do You Think We Are LP,DEEP PURPLE 30 Very Best Of 2CD (Фирм.)
item_category_id,37,58,58,58,56
shop_name,"Ярославль ТЦ ""Альтаир""","Москва ТРК ""Атриум""","Москва ТРК ""Атриум""","Москва ТРК ""Атриум""","Москва ТРК ""Атриум"""
item_category_name,Кино - Blu-Ray,Музыка - Винил,Музыка - Винил,Музыка - Винил,Музыка - CD фирменного производства


In [28]:
train.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id
count,2935849.00,2935849.00,2935849.00,2935849.00,2935849.00,2935849.00
mean,14.57,33.00,10197.23,890.85,1.24,40.00
std,9.42,16.23,6324.30,1729.80,2.62,17.10
min,0.00,0.00,0.00,-1.00,-22.00,0.00
25%,7.00,22.00,4476.00,249.00,1.00,28.00
50%,14.00,31.00,9343.00,399.00,1.00,40.00
75%,23.00,47.00,15684.00,999.00,1.00,55.00
max,33.00,59.00,22169.00,307980.00,2169.00,83.00


### Time period of the dataset

In [41]:
print(train['date'].min())
print(train['date'].max())

2013-01-01 00:00:00
2015-12-10 00:00:00


In [42]:
print(train['date'].min().date())
print(train['date'].max().date())

2013-01-01
2015-12-10


### Data Leakages
#### About data leakages I'll only be using only the "shop_id" and "item_id" that appear on the test set.

In [43]:
test_shop_ids = test['shop_id'].unique()
test_item_ids = test['item_id'].unique()

[ 5  4  6  3  2  7 10 12 28 31 26 25 22 24 21 15 16 18 14 19 42 50 49 53
 52 47 48 57 58 59 55 56 36 37 35 38 34 46 41 44 39 45]
[ 5037  5320  5233 ... 15757 19648   969]


In [45]:
print(test['shop_id'].nunique())
print(test['item_id'].nunique())

42
5100


In [46]:
# Only shops that exist in test set.
lk_train = train[train['shop_id'].isin(test_shop_ids)]

# Only items that exist in test set.
lk_train = lk_train[lk_train['item_id'].isin(test_item_ids)]

In [48]:
print(lk_train['shop_id'].nunique())
print(lk_train['item_id'].nunique())

42
4716


In [49]:
print('Data set size before leaking:', train.shape[0])
print('Data set size after leaking:', lk_train.shape[0])

Data set size before leaking: 2935849
Data set size after leaking: 1224439


### Data Cleaning
#### Only records with "item_price" > 0

In [50]:
train.shape

(2935849, 10)

In [51]:
train = train.query('item_price > 0') # .query는 조건 부합 데이터 추출, 장점은 가독성과 평즤성이지만 .loc[] 보다 속도가 느림
train.shape

(2935848, 10)